In [ ]:
# Imports
import os
import time
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import mediapipe as mp


In [ ]:
# ===============================
# GPU DETECTION & CONFIGURATION
# ===============================
print('='*60)
print('🔍 GPU DETECTION & CONFIGURATION')
print('='*60)
print(f'
TensorFlow version: {tf.__version__}')
gpus = tf.config.list_physical_devices('GPU')
print(f'Found GPUs: {gpus}')
USE_GPU = False
DEVICE = '/CPU:0'
if gpus:
    try:
        for g in gpus:
            tf.config.experimental.set_memory_growth(g, True)
        tf.config.set_visible_devices(gpus[0], 'GPU')
        USE_GPU = True
        DEVICE = '/GPU:0'
        print(
            f'✅ GPU configured: {gpus[0]}')
    except RuntimeError as e:
        print('⚠️  GPU config error:', e)
# Mixed precision (optional and beneficial on modern GPUs)
try:
    if USE_GPU:
        from tensorflow.keras import mixed_precision
        policy = mixed_precision.Policy('mixed_float16')
        mixed_precision.set_global_policy(policy)
        print('⚡ Mixed precision enabled:', policy.name)
except Exception as e:
    print('⚠️  Mixed precision not enabled:', e)
print('
Configuration complete. Using device: ', DEVICE)


GPU tips: If you run out of memory, reduce `BATCH_SIZE` or disable mixed precision. Use `nvidia-smi` in a separate terminal to monitor GPU usage.

In [ ]:
# ===============================
# MEDIAPIPE KEYPOINT EXTRACTION (GPU-friendly)
# ===============================
CSV_PATH = 'asl_mediapipe_keypoints_dataset_gpu.csv'
# Update this to your dataset location if different
DATASET_DIR = r'M:erm 9\Grad\Main\Sign-Language-Recognition-System-main\Sign-Language-Recognition-System-main\Sign_to_Sentence Project Main\ASL Letter (English)\Datasets\Asl_Sign_Data\asl_alphabet_train'
# If your ASL data is in a different path, change DATASET_DIR above
if os.path.exists(CSV_PATH):
    print('Dataset CSV already exists:', CSV_PATH)
    df = pd.read_csv(CSV_PATH)
    print(f'Samples loaded: {len(df)}')
else:
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.7)
    landmark_data = []
    labels = []
    # gather all image paths
    class_labels = sorted([d for d in os.listdir(DATASET_DIR) if os.path.isdir(os.path.join(DATASET_DIR, d))])
    all_images = []
    for label in class_labels:
        folder = os.path.join(DATASET_DIR, label)
        files = [f for f in os.listdir(folder) if f.lower().endswith(('.png','.jpg','.jpeg'))]
        for f in files:
            all_images.append((label, os.path.join(folder, f)))
    print(f'Found {len(all_images)} images across {len(class_labels)} classes')
    # process with a progress bar
    start = time.time()
    processed = 0
    skipped = 0
    for label, img_path in tqdm(all_images, desc='Extracting keypoints'):
        img = cv2.imread(img_path)
        if img is None:
            skipped += 1
            continue
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)
        if results.multi_hand_landmarks:
            # For multi-hand cases, take the first detected hand (common for single-hand dataset)
            hand_landmarks = results.multi_hand_landmarks[0]
            lm = np.array([[p.x, p.y, p.z] for p in hand_landmarks.landmark]).flatten()
            landmark_data.append(lm)
            labels.append(label)
            processed += 1
        else:
            skipped += 1
    duration = time.time() - start
    if len(landmark_data) == 0:
        print('No landmarks extracted. Check dataset or MediaPipe settings.')
        df = pd.DataFrame()
    else:
        df = pd.DataFrame(landmark_data)
        df['label'] = labels
        df.to_csv(CSV_PATH, index=False)
        print('Saved keypoints to', CSV_PATH)
        print(f'Processed: {processed}, Skipped: {skipped}, Time: {duration:.2f}s')


In [ ]:
# ===============================
# PREPROCESSING & SPLIT
# ===============================
df = pd.read_csv('asl_mediapipe_keypoints_dataset_gpu.csv')
print('Loaded samples:', len(df))
X = df.iloc[:, :-1].astype('float32').values
y = df['label'].values
encoder = LabelEncoder()
y_enc = encoder.fit_transform(y)
num_classes = len(encoder.classes_)
# stratified split
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y_enc, test_size=0.2, random_state=42, stratify=y_enc)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42, stratify=y_train_full)
# one-hot encode labels
y_train = to_categorical(y_train, num_classes=num_classes)
y_val = to_categorical(y_val, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)
# cast features
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')
print('Train/Val/Test sizes:', X_train.shape[0], X_val.shape[0], X_test.shape[0])


In [ ]:
# Efficient tf.data pipeline helper
AUTOTUNE = tf.data.AUTOTUNE
def make_dataset(features, labels, batch_size, training=True):
    ds = tf.data.Dataset.from_tensor_slices((features, labels))
    if training:
        buffer = min(len(features), 10000)
        ds = ds.shuffle(buffer_size=buffer, reshuffle_each_iteration=True)
    ds = ds.batch(batch_size).prefetch(AUTOTUNE)
    return ds


In [ ]:
# ===============================
# MODEL CREATION (GPU-OPTIMIZED)
# ===============================
tf.keras.backend.clear_session()
with tf.device(DEVICE):
    model = Sequential([
        Dense(256, activation='relu', kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(1e-4), input_shape=(X_train.shape[1],)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
        BatchNormalization(),
        Dropout(0.25),
        Dense(64, activation='relu', kernel_initializer='he_normal'),
        Dropout(0.2),
        Dense(num_classes, activation='softmax', dtype='float32')
    ])
# Use legacy Adam if mixed precision is set (works well with float16 activations)
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
print('Model will train on device:', DEVICE)


In [ ]:
# ===============================
# TRAINING (GPU-OPTIMIZED)
# ===============================
if USE_GPU:
    BATCH_SIZE = 256
else:
    BATCH_SIZE = 64
train_ds = make_dataset(X_train, y_train, BATCH_SIZE, training=True)
val_ds = make_dataset(X_val, y_val, BATCH_SIZE, training=False)
callbacks = [
    ModelCheckpoint('asl_mediapipe_mlp_model_gpu_best.h5', monitor='val_accuracy', save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7, verbose=1),
]
print('Training with batch size:', BATCH_SIZE)
start = time.time()
with tf.device(DEVICE):
    history = model.fit(train_ds, validation_data=val_ds, epochs=20, callbacks=callbacks, verbose=1)
duration = time.time() - start
print(f'Training finished in {duration:.2f}s')
model.save('asl_mediapipe_mlp_model_gpu.h5')
print('Saved model: asl_mediapipe_mlp_model_gpu.h5')


In [ ]:
# ===============================
# EVALUATION
# ===============================
model = tf.keras.models.load_model('asl_mediapipe_mlp_model_gpu.h5')
eval_batch = 256 if USE_GPU else 128
test_ds = make_dataset(X_test, y_test, eval_batch, training=False)
with tf.device(DEVICE):
    loss, acc = model.evaluate(test_ds, verbose=1)
print(f'Test Loss: {loss:.4f}  Test Accuracy: {acc*100:.2f}%')


## Webcam Inference (MediaPipe + MLP)
The following cell runs webcam inference. Press `q` to quit. It uses GPU for model prediction when available.

In [ ]:
# ===============================
# WEBCAM INFERENCE
# ===============================
encoder = LabelEncoder()
df_labels = pd.read_csv('asl_mediapipe_keypoints_dataset_gpu.csv')
encoder.fit(df_labels['label'])
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
cap = cv2.VideoCapture(0)
predicted_sentence = ''
mlp_model = tf.keras.models.load_model('asl_mediapipe_mlp_model_gpu.h5')
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            lm = np.array([[p.x, p.y, p.z] for p in hand_landmarks.landmark]).flatten().reshape(1, -1)
            input_tensor = tf.cast(lm, tf.float32)
            with tf.device(DEVICE):
                pred = mlp_model.predict(input_tensor, verbose=0)
            pred_class = np.argmax(pred)
            label = encoder.inverse_transform([pred_class])[0]
            if label == 'SPACE':
                predicted_sentence += ' '
            elif label == 'DELETE':
                predicted_sentence = predicted_sentence[:-1]
            elif label == 'NOTHING':
                pass
            else:
                predicted_sentence += label
    # draw sentence bar
    h, w, _ = frame.shape
    cv2.rectangle(frame, (0, h-60), (w, h), (0, 0, 0), -1)
    cv2.putText(frame, predicted_sentence, (10, h-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    cv2.imshow('GPU-Optimized Sign Prediction', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


### Next steps / Notes:
- If your dataset path differs, update `DATASET_DIR` (cell 5).
- To re-extract keypoints, delete the CSV `asl_mediapipe_keypoints_dataset_gpu.csv` and re-run the extraction cell.
- If you see OOM errors: lower `BATCH_SIZE` or disable mixed precision.
- You can enable `nvidia-smi -l 1` in a separate PowerShell to monitor GPU while training.